# <u>ARE WE THERE YET?</u>



&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Here I present you my creatively named Capstone project, Are We There Yet? I've always found the sports of baseball fasinating. Every play can be explained in numbers and statistics no matter how good or bad a player is. It is so fantastic that world-class athletes work with most advanced sports analytic teams to gain edge on the competition. Baseball is always evolving around the next big statistics and I very much enjoy the era of MLB Statcast. Hence, when it came to time to decide on the topic for my capstone project at BrainStation, I had no doubt that it was going to something baseball related. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;My capstone project is to predict whether a team makes a playoff or not with various forms of team statistics found on Baseball-Reference without looking at win and loss records. There are three major categories in baseball: batting, pitching and fielding. These categories have tens if not hundreds of sub-categories within them and I have chosen categories that I find seem fit for the outcomes of the teams' season. Ultimate goal of this capstone is setting a solid base in which I can branch off to other baseball related topics for future data science projects. It is very first capstone project so I figure I would start with fundamentals and build up with this.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In this workbook, I collected batting, pitching and fielding statistics from year 1969 to year 2020 that will help determining playoff teams and non-playoff teams. I output them in three different csv files which I merge them in EDA workbook. I also collected playoff history from Wikipedia and cleaned up so I could merge with other statistics csv files for modelling later on.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The tool I used for webscraping is BeautifulSoup and sources of data are Baseball-reference.com and Wikipedia.com. I appreciate the wonderful tool and websites for enabling me to dive into the world of baseball analytics and hope this is one of many more to come.

# <u>Data Collection</u>

<hr style="border:2px solid gray"> </hr>

Before moving onto actual data collection, I'd like to introduce the evolution of MLB playoff through the years as well as team expansion. This will help understand the struggle I had to go through in adjusting for precision in category 1, teams in playoff, as data from 69 to 93 was vastly different from 95 to 2020 data. 

## History of MLB Playoff

Prior to 1969, it was only 2 teams making to World Series only. I decided not to collect these data and just move on with data from 1969 and on.

From 1969 to 1993, playoff was expanded to two rounds with introduction of ALCS and NLCS. 4 teams in total

In 1981, playoff was expanded only for this year due to players' strike to 8 teams

In 1994, playoff was cancelled due to another players' strike

From 1995 to 2011, playoff expanded with restructing of divisions. MLB went through a realignment when both American and National Leagues expanded to three divisions. To avoid playoffs with odd number of divisional winners, the league added a wild card playoff spot to each league making it 8 team-playoff

From 2012 to present, playoff expanded yet again with second wild card team. One-game wild card playoff was introduced expanding to 10 team-playoff

In 2020, due to pandemic, there was a suspension of MLB season and the league expanded the playoff to 16 teams

<hr style="border:2px solid gray"> </hr>

## History of MLB Team Expansion

In 1969, mlb expands to 24 teams. Beginning of divisional play

In 1972, Washington Senators moved to Texas Rangers

In 1977, AL expands to 14 teams with additions of Toronto Blue Jays and Seattle Mariners

In 1993, NL expands to 14 teams with additions of Colorado Rockies and Florida Marlins

In 1994, Divisional Realignment and the wildcard

In 1998, MLB Expansion to 30 teams AL: Tempa Bay Devil Rays, NL: Arizona Diamondbacks

League Transfers and Division changes due to expansion

In 2005, Motnreal Expos moved to Washing Nationals

In 2013, Balance Realignment returns as Houston Astros move to AL West

In [19]:
# Import packages for webscraping
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time

In [20]:
# Create a function to grab a stat of interest
def get_stat(stat, side = 'right'):

    find_stat = soup.find_all('td', attrs = {'data-stat' : stat, 'class' : side})
    # Take out league total and average after extraction
    stat_list = [i.text for i in find_stat][: -2]

    return stat_list

In [21]:
# Instantiate df_batting. It contains all team-level batting stats

# Instantiate df_batting dataframe for batter's stats
df_batting = pd.DataFrame(columns=['team', 'batters_used', 'batter_age', 'games', 'b_runs', 'b_hits', 'doubles', 'triples', 'homeruns',
                                   'rbi', 'sb', 'cs', 'walks', 'b_strikeouts', 'batting_average', 'on_base_percentage', 'slugging',
                                   'ops', 'ops_plus', 'total_bases', 'ground_double_play', 'b_left_on_base'])

# Loop through years 1969 to 2020
for i in range(1969, 2021):
    
    # Save the URL of the webpage I want to scrape to a variable
    url = f'https://www.baseball-reference.com/leagues/MLB/{i}.shtml#all_teams_standard_batting'
    
    # Send a get request and assign the response to a variable
    response = requests.get(url)
    
    # Turn the undecoded content into a Beautiful Soup object and assign it to a variable
    soup = BeautifulSoup(response.content, 'html.parser')

    # Teams need different parameter 'th'
    find_teams = soup.find_all('th', attrs = {'data-stat': 'team_ID'})
    # Expand to list
    team_list = [i.text for i in find_teams][1: -3]

    # Batting stats that will go into dataframe
    batters_used = get_stat('batters_used')
    batter_age = get_stat('age_bat')
    games = get_stat('G')
    runs = get_stat('R')
    hits = get_stat('H')
    
    # 'doubles' and 'triples' happened to cause alot of problems with alignment curveball thrown at me 
    doubles = get_stat('2B', 'left')
    triples = get_stat('3B', 'left')
    homeruns = get_stat('HR')
    rbi = get_stat('RBI')
    sb = get_stat('SB')
    cs = get_stat('CS')
    walks = get_stat('BB')
    strikeouts = get_stat('SO')
    batting_average = get_stat('batting_avg')
    on_base_percentage = get_stat('onbase_perc')
    slugging = get_stat('slugging_perc')
    ops = get_stat('onbase_plus_slugging')
    ops_plus = get_stat('onbase_plus_slugging_plus')
    total_bases = get_stat('TB')
    gdp = get_stat('GIDP')
    left_on_base = get_stat('LOB')

    # Make dataframe with team + stats
    df = pd.DataFrame({'team': team_list, 'batters_used': batters_used, 'batter_age': batter_age, 'games': games, 'b_runs': runs,
                       'b_hits': hits, 'doubles': doubles, 'triples': triples, 'homeruns': homeruns, 'rbi': rbi,
                       'sb': sb,  'cs': cs, 'walks': walks, 'b_strikeouts': strikeouts, 'batting_average': batting_average,
                       'on_base_percentage': on_base_percentage, 'slugging': slugging, 'ops': ops,
                       'ops_plus': ops_plus, 'total_bases': total_bases, 'ground_double_play': gdp, 'b_left_on_base': left_on_base})
  
    # Add in year column to the dataframe
    df['year'] = i
    
    # Pile yearly stats into df_batting
    df_batting = df_batting.append(df)
    
    # Delay so I don't overload the server with query
    time.sleep(1)
    
    
    ############
    ### NOTE ###
    ############
### Disabled all the output as I have provided the files in zip ###
### to prevent mix up in case something goes wrong              ###

#df_batting.to_csv(r'data/batting_stats.csv', index = False)

In [22]:
# Instantiate df_pitching. It contains all team-level pitching stats

# Just like for hitting, create empty df_pitching for pitcher stats
df_pitching = pd.DataFrame(columns=['team', 'pitchers_used', 'pitcher_age', 'era', 'innings_pitched', 'p_hits', 'p_runs',
                                   'earned_runs', 'homeruns_allowed', 'walks_allowed', 'p_strikeouts', 'saves', 'hbp', 'era_plus',
                                    'whip', 'hits_per_nine', 'hr_per_nine', 'walks_per_nine', 'so_per_nine', 'so_walk', 'p_left_on_base'])

# Again, loop through same years
for i in range(1969, 2021):
    
    url = f'https://www.baseball-reference.com/leagues/MLB/{i}-standard-pitching.shtml'
    
    response = requests.get(url)

    soup = BeautifulSoup(response.text, 'html.parser')

    # Teams need different parameter 'th'
    find_teams = soup.find_all('th', attrs = {'data-stat': 'team_ID'})
    # Expand to list
    team_list = [i.text for i in find_teams][1: -3]

    # Again, pitching stats that will go into dataframe
    pitchers_used = get_stat('pitchers_used')
    pitcher_age = get_stat('age_pitch')
    era = get_stat('earned_run_avg')
    innings_pitched = get_stat('IP')
    hits = get_stat('H')
    runs = get_stat('R')
    earned_runs = get_stat('ER')
    homeruns = get_stat('HR')
    walks = get_stat('BB')
    strikeouts = get_stat('SO')
    saves = get_stat('SV')
    hbp = get_stat('HBP')
    era_plus = get_stat('earned_run_avg_plus')
    whip = get_stat('whip')
    hits_per_nine = get_stat('hits_per_nine')
    hr_per_nine = get_stat('home_runs_per_nine')
    walks_per_nine = get_stat('bases_on_balls_per_nine')
    so_per_nine = get_stat('strikeouts_per_nine')
    so_walk = get_stat('strikeouts_per_base_on_balls')
    left_on_base = get_stat('LOB')

    # Make dataframe with team + stats
    df = pd.DataFrame({'team': team_list, 'pitchers_used': pitchers_used, 'pitcher_age': pitcher_age, 'era': era, 'innings_pitched': innings_pitched,
                       'p_hits': hits, 'p_runs': runs, 'earned_runs': earned_runs, 'homeruns_allowed': homeruns,
                       'walks_allowed': walks, 'p_strikeouts': strikeouts, 'saves': saves, 'hbp': hbp, 'era_plus': era_plus, 'whip': whip,
                       'hits_per_nine': hits_per_nine, 'hr_per_nine': hr_per_nine, 'walks_per_nine': walks_per_nine,
                       'so_per_nine': so_per_nine, 'so_walk': so_walk, 'p_left_on_base': left_on_base})
   
    # Add in year column to the dataframe
    df['year'] = i
    
    # Pile yearly stats into df_pitching
    df_pitching = df_pitching.append(df)
    
    # Delay so I don't overload the server with query
    time.sleep(1)
    
#df_pitching.to_csv(r'data/pitching_stats.csv', index = False)

In [23]:
# Instantiate df_fielding. It contains all team-level fielding stats

df_fielding = pd.DataFrame(columns=['team', 'fielders_used', 'defensive_efficiency', 'errors', 'fielding_percentage', 'Rtot',])

# Loop through years that I am interested in
for i in range(1969, 2021):
    
    # Save the URL of the webpage I want to scrape to a variable
    url = f'https://www.baseball-reference.com/leagues/MLB/{i}-standard-fielding.shtml'
    
    # Send a get request and assign the response to a variable
    response = requests.get(url)
    
    # Turn the undecoded content into a Beautiful Soup object and assign it to a variable
    soup = BeautifulSoup(response.content, 'html.parser')

    # Teams need different parameter 'th'
    find_teams = soup.find_all('th', attrs = {'data-stat': 'team_ID'})
    # Expand to list
    team_list = [i.text for i in find_teams][1: -3]

    # Batting stats that will go into dataframe
    fielders_used = get_stat('fielders_used')
    def_eff = get_stat('defensive_efficiency')
    errors = get_stat('E_def')
    fielding_percentage = get_stat('fielding_perc')
    rtot = get_stat('tz_runs_total')

    # Make dataframe with team + stats
    df = pd.DataFrame({'team': team_list, 'fielders_used': fielders_used, 'defensive_efficiency': def_eff, 'errors': errors, 
                       'fielding_percentage': fielding_percentage, 'Rtot': rtot})
  
    # Add in year column to the dataframe
    df['year'] = i
    
    # Pile yearly stats into df_batting
    df_fielding = df_fielding.append(df)
    
    # Delay so I don't overload the server with query
    time.sleep(1)

# Output into csv file for later usage
#df_fielding.to_csv(r'fielding_stats.csv', index = False)

In [24]:
# This is playoff appareance table from Wikipedia as they offered it in better format for my use
playoff = pd.read_excel('data/playoff.xlsx')

In [25]:
# This requires some clean-up
playoff

,Franchise,Unnamed: 1,Years,Postseason Appearances
0,ARI,Arizona Diamondbacks,1998–present,"1999, 2001, 2002, 2007, 2011, 2017"
1,ATL,Boston Braves,1903–1952,"1914, 1948"
2,NaN,Milwaukee Braves,1953–1965,"1957, 1958"
3,NaN,Atlanta Braves,1966–present,"1969, 1982, 1991, 1992, 1993, 1995, 1996, 1997..."
4,BAL,St. Louis Browns,1903–1953,1944
5,NaN,Baltimore Orioles,1954–present,"1966, 1969, 1970, 1971, 1973, 1974, 1979, 1983..."
6,BOS,Boston Red Sox,1903–present,"1903, 1912, 1915, 1916, 1918, 1946, 1967, 1975..."
7,CHC,Chicago Cubs,1903–present,"1906, 1907, 1908, 1910, 1918, 1929, 1932, 1935..."
8,CWS,Chicago White Sox,1903–present,"1906, 1917, 1919, 1959, 1983, 1993, 2000, 2005..."
9,CIN,Cincinnati Reds,1903–present,"1919, 1939, 1940, 1961, 1970, 1972, 1973, 1975..."


In [26]:
# Fill in NaNs! 
playoff['Franchise'][2] = 'ATL'
playoff['Franchise'][3] = 'ATL'
playoff['Franchise'][5] = 'BAL'
playoff['Franchise'][17] = 'LAD'
playoff['Franchise'][20] = 'MIL'
playoff['Franchise'][22] = 'MIN'
playoff['Franchise'][26] = 'OAK'
playoff['Franchise'][27] = 'OAK'
playoff['Franchise'][33] = 'SFG'
playoff['Franchise'][37] = 'TEX'
playoff['Franchise'][40] = 'WSH'

# Check if any NaNs I missed out imputing
playoff['Franchise'].isna().sum()

0

In [27]:
# Drop rows that contain data before 1969 as playoff was World Series only (only 2 teams made the playoff)
playoff.drop([1, 2, 4, 16, 21, 25, 26, 32], inplace = True)

In [28]:
# Reset index 
playoff.reset_index(drop = True, inplace = True)

In [29]:
# Double Check
playoff

,Franchise,Unnamed: 1,Years,Postseason Appearances
0,ARI,Arizona Diamondbacks,1998–present,"1999, 2001, 2002, 2007, 2011, 2017"
1,ATL,Atlanta Braves,1966–present,"1969, 1982, 1991, 1992, 1993, 1995, 1996, 1997..."
2,BAL,Baltimore Orioles,1954–present,"1966, 1969, 1970, 1971, 1973, 1974, 1979, 1983..."
3,BOS,Boston Red Sox,1903–present,"1903, 1912, 1915, 1916, 1918, 1946, 1967, 1975..."
4,CHC,Chicago Cubs,1903–present,"1906, 1907, 1908, 1910, 1918, 1929, 1932, 1935..."
5,CWS,Chicago White Sox,1903–present,"1906, 1917, 1919, 1959, 1983, 1993, 2000, 2005..."
6,CIN,Cincinnati Reds,1903–present,"1919, 1939, 1940, 1961, 1970, 1972, 1973, 1975..."
7,CLE,Cleveland Indians,1903–present,"1920, 1948, 1954, 1995, 1996, 1997, 1998, 1999..."
8,COL,Colorado Rockies,1993–present,"1995, 2007, 2009, 2017, 2018"
9,DET,Detroit Tigers,1903–present,"1907, 1908, 1909, 1934, 1935, 1940, 1945, 1968..."


In [30]:
# Rename Seattle Pilots, Washington Senators, Montreal Expos so can be matched with stats dfs prepared earlier
playoff['Franchise'][15] = 'SEP'
playoff['Franchise'][28] = 'WSA'
playoff['Franchise'][31] = 'MON'

In [31]:
# Quick rename to match 'team' column in stats dfs
playoff.rename(columns = {'Franchise': 'team'}, inplace = True)

In [32]:
playoff

,team,Unnamed: 1,Years,Postseason Appearances
0,ARI,Arizona Diamondbacks,1998–present,"1999, 2001, 2002, 2007, 2011, 2017"
1,ATL,Atlanta Braves,1966–present,"1969, 1982, 1991, 1992, 1993, 1995, 1996, 1997..."
2,BAL,Baltimore Orioles,1954–present,"1966, 1969, 1970, 1971, 1973, 1974, 1979, 1983..."
3,BOS,Boston Red Sox,1903–present,"1903, 1912, 1915, 1916, 1918, 1946, 1967, 1975..."
4,CHC,Chicago Cubs,1903–present,"1906, 1907, 1908, 1910, 1918, 1929, 1932, 1935..."
5,CWS,Chicago White Sox,1903–present,"1906, 1917, 1919, 1959, 1983, 1993, 2000, 2005..."
6,CIN,Cincinnati Reds,1903–present,"1919, 1939, 1940, 1961, 1970, 1972, 1973, 1975..."
7,CLE,Cleveland Indians,1903–present,"1920, 1948, 1954, 1995, 1996, 1997, 1998, 1999..."
8,COL,Colorado Rockies,1993–present,"1995, 2007, 2009, 2017, 2018"
9,DET,Detroit Tigers,1903–present,"1907, 1908, 1909, 1934, 1935, 1940, 1945, 1968..."


In [33]:
# Only take the first and last columns of playoff df for my use
playoff = playoff[['team', 'Postseason Appearances']]

In [34]:
playoff

,team,Postseason Appearances
0,ARI,"1999, 2001, 2002, 2007, 2011, 2017"
1,ATL,"1969, 1982, 1991, 1992, 1993, 1995, 1996, 1997..."
2,BAL,"1966, 1969, 1970, 1971, 1973, 1974, 1979, 1983..."
3,BOS,"1903, 1912, 1915, 1916, 1918, 1946, 1967, 1975..."
4,CHC,"1906, 1907, 1908, 1910, 1918, 1929, 1932, 1935..."
5,CWS,"1906, 1917, 1919, 1959, 1983, 1993, 2000, 2005..."
6,CIN,"1919, 1939, 1940, 1961, 1970, 1972, 1973, 1975..."
7,CLE,"1920, 1948, 1954, 1995, 1996, 1997, 1998, 1999..."
8,COL,"1995, 2007, 2009, 2017, 2018"
9,DET,"1907, 1908, 1909, 1934, 1935, 1940, 1945, 1968..."


In [37]:
#playoff.to_csv(r'data/playoff_cleaned.csv')